In [ ]:
import os 

os.chdir(os.path.abspath(os.path.pardir))

In [ ]:
from app.clients.frame_finder import FrameFinder
from app.metrics.iou import IOUCalculator
from app.metrics.map import MAPCalculator
from app.metrics.recognition_metrics import VideoIdentificationEvaluator

In [ ]:
# Init frame finder

json_files=[]
folder_path = "results/aaivoninskaya avzotova_4 anaalesuslova"

for filename in os.listdir(folder_path):
    full_path = os.path.join(folder_path, filename)

    if os.path.isfile(full_path) and filename.lower().endswith(".json"):
        json_files.append(full_path)
frame_finder = FrameFinder(json_files=json_files)

In [ ]:
iou_calculator = IOUCalculator("annotations.xml", frame_finder)

stats = iou_calculator.get_summary_stats()

print("Person detection metrics:")
print(f"IoU mean: {stats['person_iou']['mean']:.3f} ± {stats['person_iou']['std']:.3f}")
print(f"Count difference: {stats['person_count_diff']['mean']:.1f}")

print("\nFace detection metrics:")
print(f"IoU mean: {stats['face_iou']['mean']:.3f} ± {stats['face_iou']['std']:.3f}")
print(f"Count difference: {stats['face_count_diff']['mean']:.1f}")

Person detection metrics:
IoU mean: 0.764 ± 0.078
Count difference: 0.1

Face detection metrics:
IoU mean: 0.633 ± 0.109
Count difference: -0.5


In [ ]:
map_calculator = MAPCalculator("annotations.xml", frame_finder, iou_threshold=0.5)

metrics = map_calculator.calculate_map()

print("Metrics:")
for class_name, value in metrics.items():
    print(f"{class_name}: {value:.4f}")

Metrics:
AP_face_bbox: 0.7712
AP_person_bbox: 0.9936
mAP: 0.8824


In [ ]:
def generate_ground_truth(frame_patterns, total_frames: int):
    pattern_frames = sorted(frame_patterns.keys())
    patterns = [frame_patterns[frame] for frame in pattern_frames]
    
    ground_truth = {}
    pattern_index = 0
    
    for frame in range(total_frames):
        if frame in frame_patterns:
            current_pattern = frame_patterns[frame]
        else:
            current_pattern = patterns[pattern_index % len(patterns)]
            pattern_index += 1
        
        ground_truth[frame] = current_pattern
    
    return ground_truth

In [ ]:
frame_patterns = {
    # 0: ["dkorolev", "mvmoiseev", "amzayakina"],
    # 0: ["ogmenkov"],
    0: ["aaivoninskaya", "avzotova_4", "anaalesuslova"],
    # 0: ["saslastnikov1"],

    
    
    
    # 15: ["ikmityushkin", "dkorolev", "mvmoiseev", "amzayakina"],
    # 30: [""dkorolev", "mvmoiseev""]
}

# total_frames = 2880
total_frames = 3600


ground_truth = generate_ground_truth(frame_patterns, total_frames)

In [ ]:
evaluator = VideoIdentificationEvaluator(
    known_identities=os.listdir("datasets/backup"),
    frame_finder=frame_finder
)

results = evaluator.process_video(ground_truth)

evaluator.print_summary(["aaivoninskaya", "avzotova_4", "anaalesuslova"])

In [16]:
import tensorflow as tf
print("Доступные GPU:", tf.config.list_physical_devices('GPU'))
print("Версия CUDA:", tf.sysconfig.get_build_info()["cuda_version"])
print("Версия cuDNN:", tf.sysconfig.get_build_info()["cudnn_version"])

Доступные GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Версия CUDA: 12.4
Версия cuDNN: 9


In [1]:
pip list | grep tensorflow

tensorflow              2.17.0
tensorflow_estimator    2.15.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU') 


2025-04-28 00:51:11.036494: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-28 00:51:11.044435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-28 00:51:11.054116: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-28 00:51:11.057170: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-28 00:51:11.064299: I tensorflow/core/platform/cpu_feature_guar

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
!nvcc --version  # Должно показывать 11.8

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:18:24_PDT_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0
